In [ ]:
import cv2
import numpy as np

In [ ]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_extractor(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face

In [ ]:
cap = cv2.VideoCapture(0)
count = 0

while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)


        file_name_path = 'C://Users//user//ws//mlops-pro//images//' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('face cropper', face)

    else:
        print("face not found")
        pass
    if cv2.waitKey(1) == 13 or count == 100:
        break
    
    
cap.release()
cv2.destroyAllWindows()
print("collecting samples complete")

In [ ]:
import tensorflow

In [ ]:
import keras

In [ ]:
from keras.applications import MobileNet

img_rows, img_cols = 224, 224 

MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

for layer in MobileNet.layers:
    layer.trainable = False

for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
def addTopModel(bottom_model, num_classes):

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [ ]:
num_classes = 3

FC_Head = addTopModel(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_dir = 'C://Users//user//ws//mlops-pro//'
validation_data_dir = 'C://Users//user//ws//mlops-pro//'


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=45,width_shift_range=0.3,height_shift_range=0.3,horizontal_flip=True,fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32
 
train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=(img_rows, img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)
 
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,target_size=(img_rows, img_cols),
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              shuffle=False)

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K 
from keras.utils import to_categorical

In [ ]:
checkpoint = ModelCheckpoint("fr_mobileNet.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)
  
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])
nb_train_samples = 200
nb_validation_samples = 140

epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)
modelnew.save('fr_mobileNet.h5')

In [ ]:
from keras.models import load_model
classifier = load_model('fr_mobileNet.h5')

In [ ]:
import os
import cv2
import numpy as np

In [ ]:
from os import listdir
from os.path import isfile, join

In [ ]:
fr_dict = {"[0]": "Akash", 
                      "[1]": "Manisha",
                      }

fr_dict_n = {"akash": "Akash", 
                      "manisha": "Manisha",
                      }
def draw_test(name, pred, im):
    person = person_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, person, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
   
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + fr_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)

In [ ]:
for i in range(0,10):
    input_im = getRandomImage('C://Users//user//ws//mlops-pro//')
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)


cv2.destroyAllWindows()